## Metodos:
Para poder concetarnos a postgre podemos crear un agente de langchain o podemos usar el archivo base de prem sql y modificaro para que fcunione con MCP
Donde MCP es Model Context Protocol que da read-only-acces a una base de datso de postgre esto se utiliza para que los LLMs puedan acceder a ella:
mas documentacion en: https://github.com/modelcontextprotocol/servers/tree/main/src/postgres



In [2]:
import pg8000
import time
from urllib.parse import urlparse
from premsql.executors.base import BaseExecutor

class PostgreSQLExecutor(BaseExecutor):
    def parse_dsn(self, dsn: str):
        parsed = urlparse(dsn)
        return {
            "user": parsed.username,
            "password": parsed.password,
            "host": parsed.hostname,
            "port": parsed.port or 5432,
            "database": parsed.path.lstrip("/")
        }

    def execute_sql(self, sql: str, dsn_or_db_path: str) -> dict:
        conn_params = self.parse_dsn(dsn_or_db_path)
        result, error = None, None
        start_time = time.time()
        
        try:
            conn = pg8000.connect(**conn_params)
            cursor = conn.cursor()
            cursor.execute(sql)
            try:
                result = cursor.fetchall()
                column_names = [desc[0] for desc in cursor.description]
            except pg8000.dbapi.ProgrammingError:
                # If no results to fetch (e.g., for INSERT)
                result = None
                column_names = None
            conn.commit()
        except Exception as e:
            error = str(e)
        finally:
            if 'cursor' in locals():
                cursor.close()
            if 'conn' in locals():
                conn.close()
            end_time = time.time()

        return {
            "result": result,
            "columns": column_names,
            "error": error,
            "execution_time": end_time - start_time,
        }


KeyboardInterrupt: 

In [8]:
import pg8000

# Establish connection
conn = pg8000.connect(
    user="admin",
    password="gnusolidario",
    host="localhost",      # or 'db' if you're inside another Docker container
    port=5432,
    database="ghdemo44"
)

# Create a cursor to execute SQL
cursor = conn.cursor()

# Example: Run a test query
cursor.execute("SELECT table_name FROM information_schema.tables WHERE table_schema = 'public';")
tables = cursor.fetchall()

print("Tables in ghdemo44:")
for table in tables:
    print(table[0])

# Clean up
cursor.close()
conn.close()


Tables in ghdemo44:
account_account_deferral
account_account_tax_rel
account_account
account_configuration_sequence
account_fiscalyear
account_account_template_tax_rel
account_fiscalyear_invoice_sequence
account_configuration_default_payment_term
account_account_type
account_configuration_default_account
account_configuration_tax_rounding
account_account_type_template
account_configuration
account_fiscalyear_line_rel
account_configuration_default_tax_rule
account_account_template
account_invoice
account_invoice-account_move_line
account_invoice_line_account_tax
account_invoice_payment_method
account_invoice_payment_term_line
account_journal
account_invoice_payment_term_line_delta
account_invoice_tax
account_journal_account
account_invoice_line
account_journal_sequence
account_journal_period__history
account_journal_type
account_invoice_payment_term
account_journal_period
account_journal_view
account_move_reconcile_write_off
account_journal_view_column
account_move
account_move_template